In [195]:
import json

import chardet as chardet
import pandas as pd
import numpy as np
import requests

In [196]:
metro_total = pd.read_excel('metro_stats.xlsx').drop(0)

In [197]:
metro_total.head()

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id,NameOfStation_en,Line_en,Year_en,Quarter_en,IncomingPassengers_en,OutgoingPassengers_en
1,Митино,Арбатско-Покровская линия,2021,I квартал,1913498,1829031,1138975996,NaN,NaN,NaN,NaN,NaN,NaN
2,Волоколамская,Арбатско-Покровская линия,2021,I квартал,1236714,1222309,1138975997,NaN,NaN,NaN,NaN,NaN,NaN
3,Строгино,Арбатско-Покровская линия,2021,I квартал,1938816,1903731,1138975999,NaN,NaN,NaN,NaN,NaN,NaN
4,Крылатское,Арбатско-Покровская линия,2021,I квартал,1849616,1818208,1138976000,NaN,NaN,NaN,NaN,NaN,NaN
5,Площадь Революции,Арбатско-Покровская линия,2021,I квартал,2324687,2319343,1138976008,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
metro_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687 entries, 1 to 1687
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   NameOfStation          1687 non-null   object
 1   Line                   1687 non-null   object
 2   Year                   1687 non-null   object
 3   Quarter                1687 non-null   object
 4   IncomingPassengers     1687 non-null   object
 5   OutgoingPassengers     1687 non-null   object
 6   global_id              1687 non-null   object
 7   NameOfStation_en       0 non-null      object
 8   Line_en                0 non-null      object
 9   Year_en                0 non-null      object
 10  Quarter_en             0 non-null      object
 11  IncomingPassengers_en  0 non-null      object
 12  OutgoingPassengers_en  0 non-null      object
dtypes: object(13)
memory usage: 171.5+ KB


In [199]:
response = requests.post(url='https://apidata.mos.ru/v1/datasets/624/features?api_key=ac106007dd68b52c51ae1062883b8187')

In [200]:
metro_entrances = pd.read_json(json.loads(json.dumps(response.content.decode())))

In [201]:
metro_entrances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   features  0 non-null      float64
 1   type      0 non-null      float64
dtypes: float64(2)
memory usage: 124.0 bytes


In [202]:
import  re
import chardet

with open("data-397-2022-10-21.json", 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']

metro_entrances = pd.read_json("data-397-2022-10-21.json", encoding=encoding)

def get_name(full_name):
    full_name = full_name.lower()
    full_name = re.sub("ё", "е", full_name)
    full_name = full_name.split(", ")[0]
    # return re.sub(r'[^\w]', ' ', full_name).split()[:2]
    # if len(full_name.split()) != 1:
    #     if full_name == full_name.split(',')[0]
    for i in ["народное ополчение", "мневники", "беговая", "электрозаводская", "киевская", "румянцево", "арбатская"]:
        if full_name.startswith(i):
            return i

    if full_name.startswith("театральная"):
        return "Охотный ряд".lower()
    if full_name.startswith("библиотека имени ленина"):
        return "боровицкая".lower()
    if full_name.startswith("улица старокачаловская"):
        return "Бульвар Дмитрия Донского".lower()

    return full_name

metro_entrances["Name"] = metro_entrances["Name"].apply(get_name)
metro_total["Name"] = metro_total["NameOfStation"].apply(get_name)
metro_entrances = metro_entrances[["Name", "Latitude_WGS84", "Longitude_WGS84"]]
metro_entrances


,Name,Latitude_WGS84,Longitude_WGS84
0,китай-город,55.757315,37.631677
1,китай-город,55.756855,37.631205
2,китай-город,55.757236,37.631951
3,китай-город,55.753098,37.633082
4,китай-город,55.753072,37.633559
...,...,...,...
1062,тверская,55.765663,37.604050
1063,тверская,55.765282,37.603986
1064,тверская,55.765304,37.604962
1065,тверская,55.764850,37.606255


In [203]:
a = list(metro_entrances["Name"].unique())
b = list(metro_total["Name"].unique())

for i in a+b:
    if i not in a:
        print(i, "not in a")
    if i not in b:
        print(i, "not in b")

варшавская not in b


In [204]:
for i in b:
    if i.startswith('Бульвар Дмитрия Донского'.lower() ):
        print(i)

бульвар дмитрия донского


In [205]:
b.index("бульвар дмитрия донского")

128

In [206]:
metro_entrances

,Name,Latitude_WGS84,Longitude_WGS84
0,китай-город,55.757315,37.631677
1,китай-город,55.756855,37.631205
2,китай-город,55.757236,37.631951
3,китай-город,55.753098,37.633082
4,китай-город,55.753072,37.633559
...,...,...,...
1062,тверская,55.765663,37.604050
1063,тверская,55.765282,37.603986
1064,тверская,55.765304,37.604962
1065,тверская,55.764850,37.606255


In [207]:
metro_total['IncomingPassengers'] = metro_total['IncomingPassengers'].astype(int)
metro_total['OutgoingPassengers'] = metro_total['OutgoingPassengers'].astype(int)
metro_total['TotalPassengers'] = metro_total['OutgoingPassengers'] + metro_total['IncomingPassengers']
metro_total

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id,NameOfStation_en,Line_en,Year_en,Quarter_en,IncomingPassengers_en,OutgoingPassengers_en,Name,TotalPassengers
1,Митино,Арбатско-Покровская линия,2021,I квартал,1913498,1829031,1138975996,NaN,NaN,NaN,NaN,NaN,NaN,митино,3742529
2,Волоколамская,Арбатско-Покровская линия,2021,I квартал,1236714,1222309,1138975997,NaN,NaN,NaN,NaN,NaN,NaN,волоколамская,2459023
3,Строгино,Арбатско-Покровская линия,2021,I квартал,1938816,1903731,1138975999,NaN,NaN,NaN,NaN,NaN,NaN,строгино,3842547
4,Крылатское,Арбатско-Покровская линия,2021,I квартал,1849616,1818208,1138976000,NaN,NaN,NaN,NaN,NaN,NaN,крылатское,3667824
5,Площадь Революции,Арбатско-Покровская линия,2021,I квартал,2324687,2319343,1138976008,NaN,NaN,NaN,NaN,NaN,NaN,площадь революции,4644030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683,Авиамоторная,Калининская линия,2022,III квартал,3400258,3570615,2389510472,NaN,NaN,NaN,NaN,NaN,NaN,авиамоторная,6970873
1684,Киевская,Кольцевая линия,2022,III квартал,3152625,3070193,2389510476,NaN,NaN,NaN,NaN,NaN,NaN,киевская,6222818
1685,Ботанический сад,Московское центральное кольцо,2022,III квартал,1002370,1225726,2389510486,NaN,NaN,NaN,NaN,NaN,NaN,ботанический сад,2228096
1686,Кутузовская,Московское центральное кольцо,2022,III квартал,1543093,1721750,2389510492,NaN,NaN,NaN,NaN,NaN,NaN,кутузовская,3264843


In [208]:
metro_total = metro_total.groupby(['Name']).mean()
metro_total

,IncomingPassengers,OutgoingPassengers,TotalPassengers
Name,,,
авиамоторная,2.007669e+06,2.093297e+06,4.100966e+06
автозаводская,1.700653e+06,1.815051e+06,3.515704e+06
академическая,2.657332e+06,2.671209e+06,5.328541e+06
александровский сад,1.278142e+06,1.286268e+06,2.564410e+06
алексеевская,2.620293e+06,2.699040e+06,5.319333e+06
...,...,...,...
электрозаводская,1.718966e+06,1.806569e+06,3.525536e+06
юго-восточная,8.646050e+05,8.242832e+05,1.688888e+06
юго-западная,3.734187e+06,3.757222e+06,7.491409e+06


In [209]:
metro_entrances = metro_entrances.groupby(['Name']).mean()

In [210]:
metro_entrances.columns

Index(['Latitude_WGS84', 'Longitude_WGS84'], dtype='object')

In [211]:

latitude = pd.Series(dtype=int, name='latitude')
longitude = pd.Series(dtype=int, name='longitude')
# metro_total['latitude'] = latitude
# metro_total['longitude'] = longitude

for name in list(metro_total.index):
    metro_total[metro_total.index == name]['latitude'] = metro_entrances[metro_entrances.index == name].Latitude_WGS84
    metro_total[metro_total.index == name]['longitude'] = metro_entrances[metro_entrances.index == name].Longitude_WGS84

metro_total

C:\Users\tohat\AppData\Local\Temp\ipykernel_4608\368928978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro_total[metro_total.index == name]['latitude'] = metro_entrances[metro_entrances.index == name].Latitude_WGS84
C:\Users\tohat\AppData\Local\Temp\ipykernel_4608\368928978.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro_total[metro_total.index == name]['longitude'] = metro_entrances[metro_entrances.index == name].Longitude_WGS84
C:\Users\tohat\AppData\Local\Temp\ipykernel_4608\368928978

,IncomingPassengers,OutgoingPassengers,TotalPassengers
Name,,,
авиамоторная,2.007669e+06,2.093297e+06,4.100966e+06
автозаводская,1.700653e+06,1.815051e+06,3.515704e+06
академическая,2.657332e+06,2.671209e+06,5.328541e+06
александровский сад,1.278142e+06,1.286268e+06,2.564410e+06
алексеевская,2.620293e+06,2.699040e+06,5.319333e+06
...,...,...,...
электрозаводская,1.718966e+06,1.806569e+06,3.525536e+06
юго-восточная,8.646050e+05,8.242832e+05,1.688888e+06
юго-западная,3.734187e+06,3.757222e+06,7.491409e+06


In [212]:
metro_total.columns

Index(['IncomingPassengers', 'OutgoingPassengers', 'TotalPassengers'], dtype='object')

In [214]:
metro_total.join(metro_entrances, on='Name')

,IncomingPassengers,OutgoingPassengers,TotalPassengers,Latitude_WGS84,Longitude_WGS84
Name,,,,,
авиамоторная,2.007669e+06,2.093297e+06,4.100966e+06,55.751692,37.717328
автозаводская,1.700653e+06,1.815051e+06,3.515704e+06,55.706967,37.659275
академическая,2.657332e+06,2.671209e+06,5.328541e+06,55.687789,37.573498
александровский сад,1.278142e+06,1.286268e+06,2.564410e+06,55.752193,37.610628
алексеевская,2.620293e+06,2.699040e+06,5.319333e+06,55.807929,37.638540
...,...,...,...,...,...
электрозаводская,1.718966e+06,1.806569e+06,3.525536e+06,55.781537,37.704738
юго-восточная,8.646050e+05,8.242832e+05,1.688888e+06,55.705605,37.817261
юго-западная,3.734187e+06,3.757222e+06,7.491409e+06,55.663670,37.483204
